In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
# Read AnnoPred PRS score file
score = pd.read_csv('C:\\Users\\lynn\\yale\\breast-t2_pT_non_inf_betas_1e-05.profile', delim_whitespace=True)
#print(score.columns)
#print(score.info())
print(score)

            FID      IID  PHENO       CNT     CNT2         SCORE
0            -1       -1     -9  12020974  2417559  1.305400e-08
1            -2       -2     -9  12507242  2182655  1.215920e-08
2            -3       -3     -9  12512616  2180018  1.817880e-08
3            -4       -4     -9  12499118  2191303  8.271810e-10
4            -5       -5     -9  12501118  2184320  7.531700e-09
...         ...      ...    ...       ...      ...           ...
487404  6026146  6026146     -9  12509134  2178735  1.159600e-08
487405  6026154  6026154     -9  12396882  2164003  6.597620e-09
487406  6026163  6026163     -9  12304420  2153836 -3.528220e-09
487407  6026178  6026178     -9  12527634  2236806  1.391440e-08
487408  6026181  6026181     -9  12512296  2194268  1.237080e-09

[487409 rows x 6 columns]


In [3]:
# only keep FID and SCORE column
score = score[['FID','SCORE']]
print(score)

            FID         SCORE
0            -1  1.305400e-08
1            -2  1.215920e-08
2            -3  1.817880e-08
3            -4  8.271810e-10
4            -5  7.531700e-09
...         ...           ...
487404  6026146  1.159600e-08
487405  6026154  6.597620e-09
487406  6026163 -3.528220e-09
487407  6026178  1.391440e-08
487408  6026181  1.237080e-09

[487409 rows x 2 columns]


In [4]:
#Read BC phenotype file
pheno = pd.read_csv('C:\\Users\\lynn\\yale\\ukbb_BC.tsv', delim_whitespace=True)
#print(pheno.columns)
#print(pheno.info())
print(pheno)

            eid     sex  age_recruit  year_birth date_recruit date_death  BC  \
0       1000019    Male           53        1954   2008-02-27        NaN   0   
1       1000022  Female           65        1942   2008-01-23        NaN   2   
2       1000035    Male           57        1952   2009-11-03        NaN   0   
3       1000046    Male           46        1960   2007-07-07        NaN   0   
4       1000054    Male           59        1949   2008-10-31        NaN   0   
...         ...     ...          ...         ...          ...        ...  ..   
502613  6026146  Female           55        1952   2008-08-30        NaN   0   
502614  6026154    Male           53        1955   2009-02-05        NaN   0   
502615  6026163  Female           57        1951   2009-02-21        NaN   0   
502616  6026178    Male           68        1941   2010-06-14        NaN   0   
502617  6026181  Female           45        1964   2009-11-12        NaN   0   

       BC_epistart BC_date_end  age_end

In [5]:
# update BC to 1 if it is > 0
pheno.loc[pheno['BC'] > 0, 'BC'] = 1

In [6]:
# only keep eid, age_end, BC, sex columns
pheno = pheno[['eid', 'age_end', 'BC', 'sex']]
print(pheno)

            eid  age_end  BC     sex
0       1000019       67   0    Male
1       1000022       63   1  Female
2       1000035       69   0    Male
3       1000046       61   0    Male
4       1000054       72   0    Male
...         ...      ...  ..     ...
502613  6026146       69   0  Female
502614  6026154       66   0    Male
502615  6026163       70   0  Female
502616  6026178       80   0    Male
502617  6026181       57   0  Female

[502618 rows x 4 columns]


In [7]:
print('control and case before cleanup\n', pheno['BC'].value_counts())

control and case before cleanup
 0    481536
1     21082
Name: BC, dtype: int64


In [8]:
# Read other phenotypes/lifestyle data
lifestyle = pd.read_csv('C:\\Users\\lynn\\yale\\ukbb_ALL_phenotype.csv', delim_whitespace=True)
#print(lifestyle.columns)
#print(lifestyle.info())
print(lifestyle)

            eid     smoke   alcohol  alcohol_frq  education hypertension  t2d  \
0       1000019     Never   Current          1.0       20.0          yes    0   
1       1000022     Never   Current          5.0       15.0          yes    0   
2       1000035     Never   Current          4.0       20.0          yes    0   
3       1000046  Previous  Previous          6.0       20.0           no    0   
4       1000054     Never   Current          4.0       20.0          yes    0   
...         ...       ...       ...          ...        ...          ...  ...   
502613  6026146     Never   Current          4.0       15.0           no    0   
502614  6026154  Previous   Current          2.0        7.0          yes    0   
502615  6026163   Current   Current          2.0        7.0           no    0   
502616  6026178     Never   Current          5.0       20.0          yes    0   
502617  6026181  Previous   Current          3.0       10.0          yes    0   

        stroke  height     

In [9]:
# convert smoke to numeric values
#print(lifestyle['smoke'].value_counts())
lifestyle.loc[lifestyle['smoke'] != 'Never', 'smoke'] = 1
lifestyle.loc[lifestyle['smoke'] == 'Never', 'smoke'] = 0
#print(lifestyle['smoke'].value_counts())

In [10]:
# convert alcohol to numeric values
#print(lifestyle['alcohol'].value_counts())
lifestyle.loc[lifestyle['alcohol'] != 'Never', 'alcohol'] = 1
lifestyle.loc[lifestyle['alcohol'] == 'Never', 'alcohol'] = 0
#print(lifestyle['alcohol'].value_counts())

In [11]:
# copnvert hypertension to numeric values
#print(lifestyle['hypertension'].value_counts())
lifestyle.loc[lifestyle['hypertension'] != 'yes', 'hypertension'] = 0
lifestyle.loc[lifestyle['hypertension'] == 'yes', 'hypertension'] = 1
#print(lifestyle['hypertension'].value_counts())

In [12]:
#join score and pheno by FID = eid
mergeddata = pd.merge(left=score, right=pheno, left_on='FID', right_on='eid')
#print(mergeddata)

In [13]:
#join merged data with lifestyle on eid
mergeddata = pd.merge(left=mergeddata, right=lifestyle, on='eid')
#print(mergeddata)

In [14]:
# clean data, removing all NaN values
cleanedData = mergeddata.loc[mergeddata['fast_glucose'].notna() & mergeddata['alcohol_frq'].notna() & mergeddata['bmi'].notna() & mergeddata['hdl'].notna() & mergeddata['total_chol'].notna() & mergeddata['total_trig'].notna()]
#filter out Male, using only Female data.
data = cleanedData.loc[cleanedData['sex'] == 'Female']


In [15]:
# Using only these columns as input data
input = data[['SCORE', 'age_end', 'smoke', 'alcohol',
       'alcohol_frq', 'hypertension', 't2d', 'stroke', 'bmi', 'fast_glucose',
       'hdl', 'total_chol', 'total_trig']]
#input = data[['SCORE', 'smoke', 'bmi']]

print('Data columns for input:\n', input.columns)
print('\nData info for input:')
print(input.info)
# using BC as output
output = data[['BC']]
print('\nData columns for output:\n', output.columns)
print('\nData info for put:')
print(output.info)

# using series as output
output = data['BC']

Data columns for input:
 Index(['SCORE', 'age_end', 'smoke', 'alcohol', 'alcohol_frq', 'hypertension',
       't2d', 'stroke', 'bmi', 'fast_glucose', 'hdl', 'total_chol',
       'total_trig'],
      dtype='object')

Data info for input:
<bound method DataFrame.info of                SCORE  age_end smoke alcohol  alcohol_frq hypertension  t2d  \
1       1.924630e-09       63     0       1          5.0            1    0   
5       6.043900e-09       74     1       1          1.0            0    0   
13      8.879510e-09       58     0       1          3.0            0    0   
14      3.588160e-09       66     0       0          6.0            0    0   
16      4.112830e-11       68     1       1          4.0            0    0   
...              ...      ...   ...     ...          ...          ...  ...   
487306  1.596320e-10       79     0       1          5.0            0    0   
487309  4.273150e-09       55     0       1          4.0            1    0   
487310  3.480860e-09       78

In [16]:
# get the values out of dataframe
X = input.values

In [17]:
# Using BC as target/output data, 0 - control, 1 - case
y = output.values

In [18]:
# split the data into training set and test set.
# 2/3 for training set, 1/3 for test set.
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                     random_state=1)

In [19]:
#count BC cases in train and test set
print('Number of cases in training set is', (y_train == 1).sum())
print('Number of cases in test set is ', (y_test == 1).sum())
print('Number of total cases is', (y == 1).sum())

Number of cases in training set is 12770
Number of cases in test set is  4257
Number of total cases is 17027


In [20]:
print('Training set shape is ',X_train.shape)
print('Test set shape is', X_test.shape)
print('Combined set shape is', X.shape)

Training set shape is  (170417, 13)
Test set shape is (56806, 13)
Combined set shape is (227223, 13)


In [96]:
#clf = MLPClassifier(random_state=1, max_iter=300, verbose=True, tol=1e-5, learning_rate_init=0.001, activation='logistic').fit(X_train, y_train)
clf = MLPClassifier(hidden_layer_sizes=(100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100), random_state=1, max_iter=300, tol=1e-5, verbose=True).fit(X_train, y_train)
#clf = MLPClassifier(random_state=1, max_iter=300, tol=1e-5, verbose=True).fit(X_train, y_train)
clf.predict_proba(X_test[:1])
clf.predict(X_test[:5, :])
clf.score(X_test, y_test)

Iteration 1, loss = 0.25119545
Iteration 2, loss = 0.23903261
Iteration 3, loss = 0.23634443
Iteration 4, loss = 0.23575161
Iteration 5, loss = 0.23443131
Iteration 6, loss = 0.23378068
Iteration 7, loss = 0.23366315
Iteration 8, loss = 0.23353687
Iteration 9, loss = 0.23287272
Iteration 10, loss = 0.23233220
Iteration 11, loss = 0.23216166
Iteration 12, loss = 0.23248810
Iteration 13, loss = 0.23156000
Iteration 14, loss = 0.23179934
Iteration 15, loss = 0.23168776
Iteration 16, loss = 0.23132142
Iteration 17, loss = 0.23099260
Iteration 18, loss = 0.23149971
Iteration 19, loss = 0.23067339
Iteration 20, loss = 0.23097304
Iteration 21, loss = 0.23070476
Iteration 22, loss = 0.23077524
Iteration 23, loss = 0.23102301
Iteration 24, loss = 0.23068542
Iteration 25, loss = 0.23038520
Iteration 26, loss = 0.23052963
Iteration 27, loss = 0.23033761
Iteration 28, loss = 0.23022655
Iteration 29, loss = 0.22985135
Iteration 30, loss = 0.22975597
Iteration 31, loss = 0.22987612
Iteration 32, los

0.934373129598986

In [97]:
y_pred = clf.predict(X_test)

In [98]:
y_pred_train = clf.predict(X_train)

In [99]:
y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [100]:
(y_pred == y_test).sum()

53078

In [101]:
(y_pred_train == y_train).sum()

159056

In [102]:
(y_pred != y_test).sum()

3728

In [103]:
((y_pred == y_test) & (y_test == 1)).sum()

800

In [104]:
((y_pred == y_test) & (y_test == 0)).sum()

52278

In [105]:
((y_pred != y_test) & (y_test == 1)).sum()

3457

In [106]:
((y_pred != y_test) & (y_pred == 1)).sum()

271

In [107]:
((y_pred_train == y_train) & (y_train == 1)).sum()

2300

In [108]:
((y_pred_train == y_train) & (y_train == 0)).sum()

156756

In [109]:
((y_pred_train != y_train) & (y_train == 1)).sum()

10470

In [110]:
((y_pred_train != y_train) & (y_train == 0)).sum()

891

In [348]:
#what if we randomize the y_test and see how this classifier behaves
#np.random.shuffle(y_test)

#y_test.fill(1)
#now check score again
#clf.score(X_test, y_test)